# Deploying a TF model as an AKS web service

In [ ]:
# set this variable to your container registry
# use the registry automatically created with the workspace as it is pre-authenticated with the workspace
registry_address = "abcxyz.azurecr.io"

In [ ]:
import azureml.core
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
print(f"Workspace: {ws.name}")

## Get your model
Your model may be stored in a repo, cloud storage, etc.
The purpose of this cell is just to retrive your model so we can register it with AzureML

In [ ]:
# Using a sample resnet50 model from tensorflow.org as an example
import os
import requests
import shutil
import tarfile
import tempfile
from io import BytesIO

model_url = "http://download.tensorflow.org/models/official/20181001_resnet/savedmodels/resnet_v1_fp32_savedmodel_NCHW_jpg.tar.gz"
archive_prefix = "./resnet_v1_fp32_savedmodel_NCHW_jpg/1538686758/"
target_folder = "resnet50_model"

if not os.path.exists(target_folder):
    response = requests.get(model_url)
    archive = tarfile.open(fileobj=BytesIO(response.content))
    with tempfile.TemporaryDirectory() as temp_folder:
        archive.extractall(temp_folder)
        shutil.copytree(os.path.join(temp_folder, archive_prefix), target_folder)

## Register the model

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path="resnet50_model", # This points to the local directory to upload.
                       model_name="resnet50_model", # This is the name the model is registered as.
                       tags={'area': "Image classification", 'type': "classification"},
                       description="Image classification trained on Imagenet Dataset",
                       workspace=ws)

print(f"model: {model.name}, description: {model.description}, version: {model.version}")

## Get target AKS Cluster

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException

gpu_cluster_name = "nc6-ic"
gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
print(f"AKS cluster: {gpu_cluster_name}")

# Pick one of the four options below to define an InferenceConfig

In [ ]:
#
# OPTION 0: use built in DEFAULT_GPU_IMAGE + apply a conda config
#
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment, DEFAULT_GPU_IMAGE # See https://github.com/Azure/AzureML-Containers/tree/master/base for default images definitions

# Defining CondaDependencies
from azureml.core.conda_dependencies import CondaDependencies
conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow-gpu==1.12.0','numpy'],
                                              pip_packages=['azureml-contrib-services', 'azureml-defaults'])

# Defininig Environment
env = Environment('tf-1.12.0-gpu')
env.docker.base_image = DEFAULT_GPU_IMAGE
env.python.conda_dependencies = conda_dependencies

# InferenceConfig
inference_config = InferenceConfig(entry_script="score.py", environment=env)
aks_config = AksWebservice.deploy_configuration()
aks_service_name ='resnet50-option0'

In [ ]:
#
# OPTION 1a: use a custom base image + apply a CondaDependencies objet
#
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.core.environment import Environment

# Defining CondaDependencies
from azureml.core.conda_dependencies import CondaDependencies
conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow-gpu==1.12.0','numpy'],
                                              pip_packages=['azureml-contrib-services', 'azureml-defaults'])

# Defining Environment
env = Environment('tf-1.12.0-gpu')
env.python.conda_dependencies = conda_dependencies
env.docker.base_image = registry_address+"/tf-1.12.0-gpu:option1"
env.inferencing_stack_version="latest"

# InferenceConfig
inference_config = InferenceConfig(entry_script="score.py", environment=env)
aks_config = AksWebservice.deploy_configuration()
aks_service_name ='resnet50-option1a'

In [ ]:
#
# OPTION 1b: use a custom base image + apply a conda.yaml environment file definition
#
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.core.environment import Environment

# Defining Environment from a conda.yaml environment file
env = Environment.from_conda_specification(name="tf-1.12.0-gpu", file_path="option1-conda.yaml")
env.docker.base_image = registry_address+"/tf-1.12.0-gpu:option1"
env.inferencing_stack_version="latest"

# InferenceConfig
inference_config = InferenceConfig(entry_script="score.py", environment=env)
aks_config = AksWebservice.deploy_configuration()
aks_service_name ='resnet50-option1b'

In [ ]:
#
# OPTION 2: use a fully configured docker image
#
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.core.environment import Environment

env = Environment('tf-1.12.0-gpu')
env.python.user_managed_dependencies=True
env.docker.base_image = registry_address+"/tf-1.12.0-gpu:option2"
env.inferencing_stack_version="latest"

inference_config = InferenceConfig(entry_script="score.py", environment=env)
aks_config = AksWebservice.deploy_configuration()

aks_service_name ='resnet50-option2'

# Deploy the model as an AKS Service using the InferenceConfig

In [ ]:
%%time
aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=gpu_cluster,
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

In [ ]:
# In case of error, you can display the service creation and boot logs
print(aks_service.get_logs())

# Test the web service
We test the web sevice by passing the test images content.

In [ ]:
%%time
import requests

# if (key) auth is enabled, fetch keys and include in the request
key1, key2 = aks_service.get_keys()

headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

# # if token auth is enabled, fetch token and include in the request
# access_token, fetch_after = aks_service.get_token()
# headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

test_sample = open('test_image.jpg', 'rb').read()
resp = requests.post(aks_service.scoring_uri, test_sample, headers=headers)
print(resp)
print(resp.text)